<a href="https://colab.research.google.com/github/sreeniketh45/hello-sree/blob/main/HackathonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers 3.0.2
!pip install --upgrade git+https://github.com/huggingface/transformers.git

In [68]:
import pandas as pd
import tensorflow as tf
import torch as t
import transformers
from transformers import pipeline

In [69]:
classifier = pipeline("sentiment-analysis")
global inputDataSetPath
inputDataSetPath = "/content/TestData.dat"

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [70]:
def readInputDataset(path):
  offers = pd.read_csv(path , sep=',' , engine='python' , header=None)
  offers.columns = [offers[0][0],offers[1][0],offers[2][0],offers[3][0],offers[4][0],offers[5][0],offers[6][0],offers[7][0]]
  offers.drop([0], axis=0, inplace=True)
  print("Scanned Input Data Set!")
  return offers

In [71]:
def filter_Records(lobName, locationName, offers):
  filteredOffers = offers[offers['Lob'].str.contains(lobName, na = False)]
  filteredOffers = filteredOffers[filteredOffers['Location'].str.contains(locationName, na = False)]
  print("Filtered the Records Based on User Selected Location and LOB")
  return filteredOffers

In [80]:
def sentimental_analysis(User_Review):
  dataFrame = [User_Review]
  candidate_labels = ['negative', 'positive']
  reviewDetails = classifier(dataFrame, candidate_labels)
  reviewDict = reviewDetails[0]
  return reviewDict

In [91]:
def getReviewsSentimentalAnalysis(filteredOffers):
  reviewpercentlist = []
  for i in filteredOffers['User_Review']:
    reviewDict = sentimental_analysis(i)
    boolean_label = reviewDict.get('label')
    percent = reviewDict.get('score')
    percent = percent*100
    if(boolean_label=='NEGATIVE'):
      percent=percent*-1
    reviewpercentlist.append(percent);
  filteredOffers['Review_Percent']= reviewpercentlist
  return filteredOffers

In [103]:
def sortDataframe(filteredOffers):
  final_dataFrame = filteredOffers.sort_values(by = ["Review_Percent","No_Of_Orders"], ascending=False)
  return final_dataFrame

In [99]:
def getOffers(lob_Name,location_Name):
  offers = readInputDataset(inputDataSetPath)
  filteredOffers = filter_Records(lob_Name,location_Name,offers)
  filteredOffers = getReviewsSentimentalAnalysis(filteredOffers)
  bestOfferDataFrame = sortDataframe(filteredOffers)
  return bestOfferDataFrame.head(5)

In [104]:
bestOfferDataFrame = getOffers('Broadband','Bengaluru')
print("Fetched the Best Offers for You! Please Find them Below : ")
bestOfferDataFrame

Ignoring args : (['negative', 'positive'],)
Ignoring args : (['negative', 'positive'],)
Ignoring args : (['negative', 'positive'],)
Ignoring args : (['negative', 'positive'],)


Scanned Input Data Set!
Filtered the Records Based on User Selected Location and LOB
Fetched the Best Offers for You! Please Find them Below : 


,Sl.No,Offer_Id,Location,No_Of_Orders,No_Of_Opportunities,No_Of_Renewals,Lob,User_Review,Review_Percent
3,3,BROD10,Bengaluru,100,120,30,Broadband,This is Excellent,99.986136
1,1,BROD40,Bengaluru,200,300,100,Broadband,This is Very Good,99.984396
2,2,BROD30,Bengaluru,100,150,30,Broadband,This Could be More Better,-99.724579
4,4,BROD70,Bengaluru,60,90,5,Broadband,Very Poor and Worst Speed,-99.980980
